# Notebook to start overviewing the data from ECG channel

## Import libraries

In [1]:
import os
import mne
import matplotlib.pyplot as plt
import neurokit2 as nk
import pandas as pd
%matplotlib qt

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in ""


# Load data form parquet

In [10]:
# read data
data = pd.read_parquet('./sub-032_task-rest_ecg.parquet')

# constants
sampling_rate = 1000

ecg_signal = data['ECG']


# Start ECG analyses

In [12]:
signals, info = nk.ecg_process(data['ECG'], sampling_rate=1000)

In [13]:
nk.ecg_plot(signals, info)

In [14]:
# Calculate HRV metrics (Time, Frequency, and Non-Linear domains)
# You pass the 'info' dictionary you just created
hrv_indices = nk.hrv(info, sampling_rate=1000, show=True)

# Look at the results
display(hrv_indices)

,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,741.570492,68.272354,18.412395,66.351952,NaN,NaN,NaN,NaN,50.985711,51.067603,...,1.293878,7.168335,1.056541,1.359464,1.250382,1.910414,1.652023,1.831503,2.400347,0.811738


In [15]:
# Calculate HRV metrics (Time, Frequency, and Non-Linear domains)
# You pass the 'info' dictionary you just created
hrv_indices = nk.hrv_nonlinear(info, sampling_rate=1000, show=True)

# Look at the results
display(hrv_indices)

,HRV_SD1,HRV_SD2,HRV_SD1SD2,HRV_S,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_PIP,HRV_IALS,HRV_PSS,...,HRV_SampEn,HRV_ShanEn,HRV_FuzzyEn,HRV_MSEn,HRV_CMSEn,HRV_RCMSEn,HRV_CD,HRV_HFD,HRV_KFD,HRV_LZC
0,36.110249,89.296056,0.404388,10130.074273,2.472873,4.712583,883.271199,0.360656,0.347973,0.378641,...,1.293878,7.168335,1.056541,1.359464,1.250382,1.910414,1.652023,1.831503,2.400347,0.811738
